# Desafio (Parte 2) - Recomendação de Produtos

## Case baseado no desafio do Kaggle: "Instacart Market Basket Analysis"

Esse desafio é baseado em uma competição do Kaggle ([https://www.kaggle.com](https://www.kaggle.com)) de 2017, onde é pedido para fazer a predição de se um produto será comprado por dado usuário no carrinho atual, baseado em suas compras anteriores.

Link para o desafio: [https://www.kaggle.com/c/instacart-market-basket-analysis](https://www.kaggle.com/c/instacart-market-basket-analysis)

## Origem dos Dados:

Os dados mostrados nesse desafio são uma amostra extraída do dataset disponibilizado gratuitamente pela empresa Instacart ([https://www.instacart.com/](https://www.instacart.com/)) **\***. São dados reais do banco de dados da empresa, que foram anonimizados para o uso por pesquisadores e em competições de Data Science.

 
###### * “The Instacart Online Grocery Shopping Dataset 2017”, Accessed from [https://www.instacart.com/datasets/grocery-shopping-2017](https://www.instacart.com/datasets/grocery-shopping-2017) on March 10, 2017.

___

# Imports

In [ ]:
import numpy as np
import os
import pandas as pd

In [ ]:
""" habilitando plots no notebook """
%matplotlib inline

In [ ]:
""" plot libs """
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
""" Configurando o Matplotlib para o modo manual """
plt.interactive(False)

# Objetivo :

Analisar o dataset (o mesmo construído na Aula 07) e responder às questões em cada item usando gráficos.

# Importante:

Todos os gráficos devem conter:
* Título
* Rótulos do Eixo X e do Eixo Y

Se mais de um gráfico for plotado em uma única janela, eles devem ter:
* Cores diferentes
* Legenda
* Transparência (na sobreposição)

# Dataset:

O dataset utilizado nesse desafio é um sub-conjunto daquele construído na Aula 7. Para essa aula, será necessário carregar apenas duas colunas: `user_id` e `product_name`.

## Carregando os dados

In [ ]:
""" Nota sobre o Warning: 
    O código abaixo levanta um Warning gerado por uma chamada interna do pandas conflitando com o Numpy.
    Não há necessidade de tratar esse Warning no momento, o comportamento do 'read_csv' é funcional para o que queremos.
"""
dataset = pd.read_csv(
    os.path.join("data", "dataset.csv"),
    usecols=["user_id", "product_name"],
    sep=",",
    index_col=0,
    encoding="utf-8"
)

## Extração dos dados específicos

In [ ]:
product_count = dataset.groupby("user_id").product_name.count()

In [ ]:
product_count.describe()

In [ ]:
product_count.head(10)

# Problemas

## Conceito: Recomendação Simples de Produtos

Um sistema de recomendação bem rudimentar pode ser construído usando a teoria de [`Collaborative Filtering`](https://en.wikipedia.org/wiki/Collaborative_filtering). 

Na figura abaixo está mostrado um exemplo simples:

![Collaborative Filtering](images/Collaborative_filtering.gif)

A idéia principal é representar um `usuário` pelo seu `perfil de compras`, ou seja, o usuário será representado por **todos os produtos** que ele já comprou.

## Construção do Dataset

Como serão usados métodos computacionais muito pesados, será necessário reduzir bastante a quantidade de dados utilizados. Dessa forma, serão usados apenas os **1% dos dados de usuários que compraram mais produtos** do dataset.

### Selecionando os usuários

In [ ]:
product_count.describe(percentiles=[.1, .25, .5, .75, .99])

In [ ]:
selected = product_count[product_count >= 37].index
selected.shape

In [ ]:
dataset.loc[selected].shape

In [ ]:
dataset.loc[selected].head()

### Criando a Tabela de Ocorrências

Uma `Tabela de Ocorrências` relaciona quantos produtos um usuário comprou **para cada produto**. 

Exemplo:

a) Dado Original:

| user_id | product_name  |
|:------- |:------------- |
| 12345   | Ground Coffee |
| 12345   | White Sugar   |
| 12345   | Bread         |
| 67890   | Ground Coffee |
| 67890   | A Type Milk   |
| 67890   | Vodka         |
| 67890   | Fudge Brownie |

b) Tabela de Ocorrências:

| user_id | Ground Coffee | White Sugar   | Bread         | A Type Milk   | Vodka         | Fudge Brownie |
|:------- |:------------- |:------------- |:------------- |:------------- |:------------- |:------------- |
| 12345   | 1             | 1             | 1             | 0             | 0             | 0             |
| 67890   | 1             | 0             | 0             | 1             | 1             | 1             |




In [ ]:
temp = dataset.loc[selected].reset_index()
users = temp.user_id
prods = temp.product_name

In [ ]:
user_prod = pd.crosstab(users, prods)
user_prod.head(10)

### Redução de Dimensionalidade

A `Tabela de Ocorrências` é uma matriz muito grande e muito **esparsa**, i.e. possui muitas colunas e pouca informação. Por pouca informação pode-se entender que menos de 1% da matriz é composta por elementos **não zeros**.

Para esse tipo de dado, em que é necessária muita memória e processamento para se extrair informações relevantes, usam-se técnicas de **redução de dimensionalidade**. Essas técnicas têm como objetivo condensar (ou mesmo comprimir) a informação esparsa em uma representação muito menor e densa.

Nesse desafio será utilizado a técnica linear de extração de componentes principais, ou `Principal Component Analysis` (`PCA`). Para saber mais sobre essa ferramenta, veja os links abaixo:

- [Teoria do PCA (Wikipédia)](https://en.wikipedia.org/wiki/Principal_component_analysis)
- [Implementação do Scikit-Learn](http://scikit-learn.org/stable/modules/generated/sklearn.decomposition.PCA.html)

A figura abaixo mostra um exemplo de PCA aplicado a um conjunto de dados de 2 variáveis (`x` e `y`). Pode-se observar que os novos eixos encontrados descrevem muito mais facilmente a distribuição de dados do que os eixos de `x` e `y` originais.

![GaussianScatterPCA](images/GaussianScatterPCA.svg)


Sem entrar muito em detalhes técnicos de implementação do PCA, essa técnica descorrelaciona (torna **linearmente independentes**) os dados originais e os ordena do mais importante (i.e. aquele que carrega **mais informação**) para o menos; com isso, pode-se escolher manter apenas as nosvas dimensões que **melhor explicam** os dados e jogar fora o resto. 

Nesse desafio, vamos reduzir a tabela de dados das `12.962` colunass originais para apenas `100` usando a implementação de PCA do `scikit-learn`. 

In [ ]:
from sklearn.decomposition import PCA

user_prod_pca = pd.DataFrame(
    index=user_prod.index,
    data=PCA(100).fit_transform(user_prod)
)
user_prod_pca.head()

Para toda a Parte 2 do desafio, será usada a tabela gerada por PCA `user_prod_pca`. 

## A) Visualização e Seleção dos Usuários

Como o PCA ordena as _features_ da mais importante para a menos importante, as duas primeiras colunas da tabela `user_prod_pca` podem ser usadas para visualização dos usuários do sistema. 

Como temos muitos usuários ainda (`1.194`), será necessário escolher alguns poucos para observar a `similaridade` entre eles.   



### A.1) Visualização

Crie um `scatter plot` que mostra a representação 2D de todos os usuários em `user_prod_pca` usando as colunas `0` para  o eixo `x` e a coluna `1` para o eixo `y`. 

Esse gráfico deve ter as seguintes características:
* Tamanho da Figura: 15 x 15
* Título e Rótulos para os eixos X e Y
* Tamanho do símbolo no mínimo igual a 60
* Transparência, para verificar dados que eventualmente se sobreponham.

In [ ]:
""" Escreva a a Solução Aqui """    

### A.2) Seleção

Crie outro `scatter plot` que mostra apenas uma parte do domínio onde há uma maior concentração de usuários; esse gráfico será parecido com um **zoom** nessa parte do domínio. 

Sugestão do corte:
- `-0.7 < X < -0.5`
- `-0.6 < Y < -0.4`

Esse gráfico deve ter as mesmas características do anterior e ainda mostrar, para cada usuário, o texto contendo seu `user_id`.

In [ ]:
""" Escreva a a Solução Aqui """    

## B) Similaridade usando Correlação entre Usuários

Para saber se um usuário é `similar` a outro, pode-se utilizar várias métricas de similaridade. Como o método de redução de dimensionalidade escolhido é linear, pode-se usar o `coeficiente de correlação de Pearson` entre as representações de dimensão `100` dos usuários como métrica de similaridade. Para saber mais sobre a correlação de Pearson, veja [este link da Wikipédia](https://en.wikipedia.org/wiki/Pearson_correlation_coefficient).

A interpretação do coeficiente `r` como métrica de similaridade é a seguinte:
* Se `|r|` é próximo de `1`, os elementos são **muito similares**;
* Se `|r|` é próximo de `0`, os elementos são **muito diferentes**.


### B.1) Visualização da Correlação

Selecione `10` usuários do `item A`, dentre próximos e distantes, e visualize a `correlação` entre todos eles.

Dica 1: use um método que mostre a correlação de todos contra todos em vez de criar uma visualização para cada dupla.

Dica 2: A maioria das funções de correlação enxergam os dados por coluna em vez de linhas; tente transpor a sua matriz caso o resultado esteja estranho. 

In [ ]:
""" Escreva a a Solução Aqui """    

### B.2) Verificando a Similaridade das Compras dos Usuários

Escolha dois usuários com uma correlação alta e verifique os produtos que ambos compraram. Faça o mesmo depois para usuários de correlação baixa.

In [ ]:
""" Escreva a a Solução Aqui """    